In [1]:
from math import ceil
import pandas as pd

Ws = """70 99 17 39 69 63 22 94 73 47 31 62 82 90 92 91 57 15 21 57
74 91 47 51 31 21 37 40 54 30 98 25 81 16 16 02 31 39 96 04
38 80 18 21 70 62 12 79 77 85 36 04 76 83 07 59 57 44 99 11"""

def min_count(W, C):
    return ceil(sum(W) / C)

def NFA(W, C, sort=False):
    NFA.comparisons = 0
    def _key_sort(x):
        NFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    NFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    for i, w in enumerate(W[1:]):
        NFA.comparisons += 1
        if w < C - Cs[-1]:
            Cs[-1] += w
        else:
            Cs.append(w)
        NFA.history.append(dict(
            weight_index=i + 2,
            weight=w,
            container_index=len(Cs),
        ))
    return len(Cs), NFA.comparisons, pd.pivot_table(pd.DataFrame(NFA.history), index='container_index', columns='weight_index', values='weight').fillna('')

def FFA(W, C, sort=False):
    FFA.comparisons = 0
    def _key_sort(x):
        FFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    FFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    for j, w in enumerate(W[1:]):
        b = True
        for i in range(len(Cs)):
            if not b:
                continue
            FFA.comparisons += 1
            if w < C - Cs[i] and b:
                Cs[i] += w
                b = False
        if b:
            Cs.append(w)
        FFA.history.append(dict(
            weight_index=j + 2,
            weight=w,
            container_index=i + 1,
        ))
    return len(Cs), FFA.comparisons, pd.pivot_table(pd.DataFrame(FFA.history), index='container_index', columns='weight_index', values='weight').fillna('')

def WFA(W, C, sort=False):
    WFA.comparisons = 0
    def _key_sort(x):
        WFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    WFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    for i, w in enumerate(W[1:]):
        worst = Cs.index(min(Cs))
        WFA.comparisons += (len(Cs) + worst + 1)
        if w < C - Cs[worst]:
            Cs[worst] += w
        else:
            worst = len(Cs)
            Cs.append(w)
        WFA.history.append(dict(
            weight_index=i + 2,
            weight=w,
            container_index=worst + 1,
        ))
    return len(Cs), WFA.comparisons, pd.pivot_table(pd.DataFrame(WFA.history), index='container_index', columns='weight_index', values='weight').fillna('')

def BFA(W, C, sort=False):
    BFA.comparisons = 0
    def _key_sort(x):
        BFA.comparisons += 1
        return x
    if sort:
        W = sorted(W, key=_key_sort, reverse=True)
    BFA.history = [dict(
        weight_index=1,
        weight=W[0],
        container_index=1,
    )]
    W = W[:]
    Cs = W[:1]
    def _greater(a, b):
        BFA.comparisons += 1
        return a > b
    for i, w in enumerate(W[1:]):
        prepared = [c if _greater(C - c, w) else -1 for c in Cs]
        worst = prepared.index(max(prepared))
        BFA.comparisons += 1 + worst
        if prepared[worst] == -1:
            worst = len(Cs)
            Cs.append(w)
        else:
            Cs[worst] += w
        BFA.history.append(dict(
            weight_index=i + 2,
            weight=w,
            container_index=worst + 1,
        ))
    return len(Cs), BFA.comparisons, pd.pivot_table(pd.DataFrame(BFA.history), index='container_index', columns='weight_index', values='weight').fillna('')


Ws = [list(map(int, w.split(' '))) for w in Ws.split('\n')]
C = 100

res_count = list()
res_diff = list()

for i, W in enumerate(Ws + [Ws[0] + Ws[1] + Ws[2]]):
    row1 = dict()
    row2 = dict()
    
    row1['Аналитический расчет'] = min_count(W, C)
    
    row1['NFA'] = NFA(W, C)[0]
    row1['FFA'] = FFA(W, C)[0]
    row1['WFA'] = WFA(W, C)[0]
    row1['BFA'] = BFA(W, C)[0]
    row2['BFA']  = BFA(W, C)[1]
    row2['NFA']  = NFA(W, C)[1]
    row2['FFA']  = FFA(W, C)[1]
    row2['WFA']  = WFA(W, C)[1]
    
    row1['NFA sorted'] = NFA(W, C, True)[0]
    row1['FFA sorted'] = FFA(W, C, True)[0]
    row1['WFA sorted'] = WFA(W, C, True)[0]
    row1['BFA sorted'] = BFA(W, C, True)[0]
    row2['FFA sorted']  = FFA(W, C, True)[1]
    row2['WFA sorted']  = WFA(W, C, True)[1]
    row2['NFA sorted']  = NFA(W, C, True)[1]
    row2['BFA sorted']  = BFA(W, C, True)[1]
    
    res_count.append(row1)
    res_diff.append(row2)

In [2]:
print('NFA')
NFA(Ws[0], C)[2]

NFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,70,,,,,,,,,,,,,,,,,,,
2,,99,,,,,,,,,,,,,,,,,,
3,,,17,39,,,,,,,,,,,,,,,,
4,,,,,69,,,,,,,,,,,,,,,
5,,,,,,63,22,,,,,,,,,,,,,
6,,,,,,,,94,,,,,,,,,,,,
7,,,,,,,,,73,,,,,,,,,,,
8,,,,,,,,,,47,31,,,,,,,,,
9,,,,,,,,,,,,62,,,,,,,,


In [3]:
print('FFA')
FFA(Ws[0], C)[2]

FFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,70,99,,,,,,,,,,,,,,,,,,
2,,,17,39,,,,,,,,,,,,,,,,
3,,,,,69,,,,,,,,,,,,,,,
4,,,,,,63,,,,,,,,,,,,,,
5,,,,,,,22,94,,,,,,,,,,,,
6,,,,,,,,,73,,,,,,,,,,,
7,,,,,,,,,,47,,,,,,,,,,
8,,,,,,,,,,,31,62,,,,,,,,
9,,,,,,,,,,,,,82,,,,,,,


In [4]:
print('WFA')
WFA(Ws[0], C)[2]

WFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,70,,17,,,,,,,,,,,,,,,,,
2,,99,,,,,,,,,,,,,,,,,,
3,,,,39,,,22,,,,,,,,,,,,21,
4,,,,,69,,,,,,,,,,,,,,,
5,,,,,,63,,,,,,,,,,,,,,
6,,,,,,,,94,,,,,,,,,,,,
7,,,,,,,,,73,,,,,,,,,,,
8,,,,,,,,,,47,31,,,,,,,,,
9,,,,,,,,,,,,62,,,,,,,,


In [5]:
print('BFA')
BFA(Ws[0], C)[2]

BFA


weight_index,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
container_index,,,,,,,,,,,,,,,,,,,,
1,70,,17,,,,,,,,,,,,,,,,,
2,,99,,,,,,,,,,,,,,,,,,
3,,,,39,,,,,,47,,,,,,,,,,
4,,,,,69,,22,,,,,,,,,,,,,
5,,,,,,63,,,,,31,,,,,,,,,
6,,,,,,,,94,,,,,,,,,,,,
7,,,,,,,,,73,,,,,,,,,,21,
8,,,,,,,,,,,,62,,,,,,,,
9,,,,,,,,,,,,,82,,,,,15,,


In [6]:
print('Количество контейнеров:')
pd.DataFrame(res_count)

Количество контейнеров:


,BFA,BFA sorted,FFA,FFA sorted,NFA,NFA sorted,WFA,WFA sorted,Аналитический расчет
0,14,14,15,14,15,16,15,14,12
1,10,10,10,10,12,11,10,10,9
2,13,12,13,12,15,14,13,12,11
3,35,33,35,33,41,39,37,33,31


In [7]:
print('Сложность:')
pd.DataFrame(res_diff)

Сложность:


,BFA,BFA sorted,FFA,FFA sorted,NFA,NFA sorted,WFA,WFA sorted
0,180,260,121,163,19,39,221,354
1,165,227,86,126,19,39,191,262
2,167,235,90,128,19,39,232,316
3,1420,2044,851,1093,59,119,2054,2570
